# imports and installs

In [ ]:
# standard python imports
import os

In [ ]:
# earth engine api import and authentication
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
# pip installable packages
try:
    import geemap
except:
    !pip install geemap
    import geemap

In [ ]:
# autoreload all python modules before executing code block
%load_ext autoreload
%autoreload 2

In [ ]:
# install and import my packages
this_dir = os.getcwd()
msslib_dir = '~/school/msslib/'
sequencer_dir = '~/school/sequencer/'
mfd_dir = '~/school/MSSForestDisturbances/'

packages = [msslib_dir, sequencer_dir, mfd_dir]

for package in packages:
    %cd {package}
    ! pip install .
    
%cd {this_dir}
    
from msslib import msslib
import sequencer
import mfd

# Test

In [ ]:
def get_valid_burns(events):
    years = events.distinct("year").aggregate_array("year")

    def _valid(year):
        curr_events = events.filter(ee.Filter.eq("year", year))
        exclusively_burnt = curr_events.reduce(ee.Reducer.allNonZero())
        burnt_often = curr_events.reduce(ee.Reducer.sum()).gt(1)
        return exclusively_burnt.Or(burnt_often).selfMask()

    result = ee.ImageCollection.fromImages(years.map(_valid))
    return result.reduce(ee.Reducer.anyNonZero())

def postprocess(event):
    event = mfd.postprocess.postprocess(event)
    event = mfd.postprocess.squash_extra_classes(event)
    event = mfd.postprocess.set_true_date(event)
    return event

In [ ]:
grid_split_qc = ee.FeatureCollection('users/boothmanrylan/gridSplitForestedQC')

grid_cell = grid_split_qc.filter(ee.Filter.eq("id", 48)).first().geometry()


In [ ]:
quebec_fires = ee.ImageCollection("users/boothmanrylan/MSSForestDisturbances/test2")

events = mfd.postprocess.filter_and_clip(quebec_fires, grid_cell)

processed_events = events.map(postprocess)

events_carried_forward = mfd.postprocess.carry_observations_forward(processed_events)
burns = sequencer.match_sequence(events_carried_forward, ee.List([1, 1, 1]))

permanent_burns = mfd.postprocess.get_burn_year_spread(processed_events).gt(5)

valid_burns = get_valid_burns(processed_events).And(permanent_burns.Not())
all_burns = processed_events.reduce(ee.Reducer.anyNonZero())

In [ ]:
binary_vis = {'min': 0, 'max': 1}

Map = geemap.Map()
Map.addLayer(grid_split_qc, {}, 'Grid Split QC')
Map.addLayer(grid_cell, {}, 'Grid Cell of Interest')
Map.addLayer(valid_burns, binary_vis, 'Valid Burns')
Map.addLayer(all_burns, binary_vis, 'All Burns')
Map.addLayer(burns, binary_vis, "Burns")
Map.centerObject(grid_cell, 10)

Map